In [5]:
# Install a pip package in the current Jupyter kernel - REMOVE IN PROD
import sys
!{sys.executable} -m pip install gtfparse

  Created wheel for gtfparse: filename=gtfparse-1.2.0-py3-none-any.whl size=11881 sha256=610294e06e765a8f59c3b8ed7ab267f2f62a13c364d779c5e3160a0299a403aa
  Stored in directory: /home/jovyan/.cache/pip/wheels/2d/81/83/85c4108d53b4b2ad3cee5d6f9b7ec612554e132202b8020359
Successfully built gtfparse


In [29]:
# Import libs
import os.path
from os import path
import gzip
import pandas as pd
import urllib.request
from gtfparse import read_gtf

In [31]:
# Setup
data_path = '../data'
gtf_path_gz = data_path + '/Mus_musculus.GRCm38.99.gtf.gz'
gtf_path = data_path + '/Mus_musculus.GRCm38.99.gtf'

In [33]:
# Download initial datasets and decompress
if path.exists(gtf_path_gz) is False: 
    urllib.request.urlretrieve('ftp://ftp.ensembl.org/pub/release-99/gtf/mus_musculus/Mus_musculus.GRCm38.99.gtf.gz', gtf_path_gz)
    input = gzip.GzipFile(gtf_path_gz, 'rb')
    s = input.read()
    input.close()
    output = open(gtf_path, 'wb')
    output.write(s)
    output.close()

In [8]:
#df = read_gtf("ftp://ftp.ensembl.org/pub/release-99/gtf/mus_musculus/Mus_musculus.GRCm38.99.gtf.gz")


In [9]:




output = open("/Users/joe/xxtest", 'wb')
output.write(s)
output.close()

print("done")

,#!genome-build GRCm38.p6
0,#!genome-version GRCm38
1,#!genome-date 2012-01
2,#!genome-build-accession NCBI:GCA_000001635.8
3,#!genebuild-last-updated 2019-09
4,1\thavana\tgene\t3073253\t3074322\t.\t+\t.\tge...
...,...
1868329,JH584295.1\tensembl\tCDS\t708\t752\t.\t-\t2\tg...
1868330,JH584295.1\tensembl\texon\t565\t633\t.\t-\t.\t...
1868331,JH584295.1\tensembl\tCDS\t565\t633\t.\t-\t2\tg...
1868332,JH584295.1\tensembl\texon\t66\t109\t.\t-\t.\tg...
